In [155]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master('local') \
    .appName('myAppName') \
    .config('spark.executor.memory', '5gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

sc = spark.sparkContext

# using SQLContext to read parquet file
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [156]:
startup=spark.read.csv("F:/Datasets/startup.csv",inferSchema=True,header=True)
consumerInternet=spark.read.parquet("F:/Datasets/consumerInternet.parquet")
data=startup.unionAll(consumerInternet)
data.printSchema()
data.show()

root
 |-- Sr_No: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Startup_Name: string (nullable = true)
 |-- Industry_Vertical: string (nullable = true)
 |-- SubVertical: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Investors_Name: string (nullable = true)
 |-- InvestmentnType: string (nullable = true)
 |-- Amount_in_USD: string (nullable = true)
 |-- Remarks: string (nullable = true)

+-----+----------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------------+-------+
|Sr_No|      Date|        Startup_Name|   Industry_Vertical|         SubVertical|         City|      Investors_Name|     InvestmentnType|Amount_in_USD|Remarks|
+-----+----------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------------+-------+
|    1|09/01/2020|              BYJU’S|              E-Tech|          E-learning|    Bengalu

In [157]:
data.createOrReplaceTempView("data")

In [158]:
PuneStartups = spark.sql("SELECT COUNT(Startup_Name) FROM data WHERE City = 'Pune'")

In [159]:
PuneStartups.show()

+-------------------+
|count(Startup_Name)|
+-------------------+
|                105|
+-------------------+



In [160]:
Startups = spark.sql("SELECT COUNT(Startup_Name) FROM data WHERE InvestmentnType like 'Seed%/%' AND City = 'Pune'")

In [161]:
Startups.show()

+-------------------+
|count(Startup_Name)|
+-------------------+
|                  6|
+-------------------+



In [176]:
from pyspark.sql.functions import *
PuneStartups = spark.sql("SELECT * FROM data WHERE City = 'Pune'")
newDf = PuneStartups.withColumn('Amount_in_USD', regexp_replace('Amount_in_USD', 'N/A', '0'))
newDf.show()

+-----+----------+----------------+-----------------+--------------------+----+--------------------+--------------------+-------------+-------+
|Sr_No|      Date|    Startup_Name|Industry_Vertical|         SubVertical|City|      Investors_Name|     InvestmentnType|Amount_in_USD|Remarks|
+-----+----------+----------------+-----------------+--------------------+----+--------------------+--------------------+-------------+-------+
|    8|12/12/2019|          Ecozen|       Technology|            Agritech|Pune|Sathguru Catalyze...|            Series A|    60,00,000|   null|
|   25|12/11/2019|     SuperGaming|      Video Games|Social gaming pla...|Pune|     Dream Incubator|        Seed Funding|    13,00,000|   null|
|   36|05/09/2019|FPL Technologies|          FinTech|  Financial Services|Pune|Matrix Partners I...|        Maiden Round|    45,00,000|   null|
|  166|01/08/2018|       HealthFin|          Finance|Patient Financing...|Pune|Axilor, Sprout Ve...| Seed/ Angel Funding|            0| 

In [208]:
from pyspark.sql.types import IntegerType
newDf.createOrReplaceTempView('newDf')
amount = newDf.withColumn("Amount_in_USD",newDf['Amount_in_USD'].cast(IntegerType()))
amount.createOrReplaceTempView('amount_table')
spark.sql("select sum(Amount_in_USD) from amount_table").show()

+------------------+
|sum(Amount_in_USD)|
+------------------+
|                 0|
+------------------+



In [192]:
TopFive = spark.sql("SELECT Industry_Vertical, COUNT(Startup_Name) FROM data GROUP BY Industry_Vertical ORDER BY COUNT(Startup_Name) DESC LIMIT 5")

In [193]:
TopFive.show()

+-----------------+-------------------+
|Industry_Vertical|count(Startup_Name)|
+-----------------+-------------------+
|Consumer Internet|                941|
|       Technology|                478|
|        eCommerce|                186|
|              nan|                171|
|       Healthcare|                 70|
+-----------------+-------------------+



In [219]:
TopInvestor = spark.sql("SELECT EXTRACT(Y FROM To_DATE(Date,'dd-MM-yyyy')) AS Date, Investors_Name FROM data")
TopInvestor

DataFrame[Date: int, Investors_Name: string]

In [220]:
TopInvestor.show()

+----+--------------------+
|Date|      Investors_Name|
+----+--------------------+
|null|Tiger Global Mana...|
|null|Susquehanna Growt...|
|null|Sequoia Capital I...|
|null|      Vinod Khatumal|
|null|Sprout Venture Pa...|
|null|   Chiratae Ventures|
|null|       Ant Financial|
|null|Sathguru Catalyze...|
|null|Ping An Global Vo...|
|null|Mumbai Angels, Ra...|
|null|SAIF Partners, Sp...|
|null|Paytm, NPTK, Sabr...|
|null|  Vertex Growth Fund|
|null|                null|
|null| Ruizheng Investment|
|null|Manipal Education...|
|null|SoftBank Vision Fund|
|null|Sequoia, CapitalG...|
|null|Sauce.vc, Rainfor...|
|null|Prime Venture Par...|
+----+--------------------+
only showing top 20 rows

